<a href="https://colab.research.google.com/github/TheAmirHK/QuantumComputation/blob/main/QuantumCryptography/QuantumCryptoKey_withError.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np

In [35]:
def generate_random_bits(n):
    """Generate n random bits."""

    return np.random.randint(2, size=n)

In [36]:
def generate_random_bases(n):
    """Generate n random bases (0 for standard, 1 for Hadamard)."""

    return np.random.randint(2, size=n)

In [37]:
def encode_qubits(bits, bases):
    """Encode bits into 'qubits' using given bases."""

    return [(bit, base) for bit, base in zip(bits, bases)]

In [38]:
def measure_qubits(qubits, measurement_bases):
    """Measure qubits in Bob's randomly chosen bases."""
    measured_bits = []
    for (bit, base), measure_base in zip(qubits, measurement_bases):
        if base == measure_base:
            measured_bits.append(bit)
        else:
            measured_bits.append(np.random.randint(2))
    return measured_bits

In [39]:
def sift_key(alice_bases, bob_bases, alice_bits, bob_bits):
    """Sift key by keeping only bits where bases match."""
    key = [alice_bits[i] for i in range(len(alice_bits)) if alice_bases[i] == bob_bases[i]]
    return key

In [40]:
# here i introduce some arbitary error in the key

def introduce_errors(bits, error_rate=0.1):
    """Simulate errors in transmission with a given probability."""
    noisy_bits = bits.copy()
    for i in range(len(bits)):
        if np.random.rand() < error_rate:
            noisy_bits[i] = 1 - noisy_bits[i]
    return noisy_bits

In [41]:
def error_detection(alice_bits, bob_bits):
    """Detect errors by checking random bits."""

    sample_size = min(len(alice_bits), 10)
    indices = np.random.choice(len(alice_bits), sample_size, replace=False)

    errors = sum(1 for i in indices if alice_bits[i] != bob_bits[i])
    error_rate = errors / sample_size

    print(f"Error Check: {errors}/{sample_size} mismatched")
    if error_rate > 0.3:
        print("Too many errors! Key is unsafe. Restart BB84.")
        return False
    return True

In [42]:
def correct_errors(alice_bits, bob_bits):
    """Fix mismatched bits using simple majority correction."""
    for i in range(len(alice_bits)):
        if alice_bits[i] != bob_bits[i]:
            bob_bits[i] = alice_bits[i]
    return bob_bits

In [43]:
def bb84_protocol(n=50):
    """Run the full BB84 quantum key distribution simulation."""
    alice_bits = generate_random_bits(n)
    alice_bases = generate_random_bases(n)

    qubits = encode_qubits(alice_bits, alice_bases)

    bob_bases = generate_random_bases(n)
    bob_bits = measure_qubits(qubits, bob_bases)

    shared_key = sift_key(alice_bases, bob_bases, alice_bits, bob_bits)

    return shared_key

In [51]:
# Generate BB84 key
n = 10
alice_bits = generate_random_bits(n)
alice_bases = generate_random_bases(n)
bob_bases = generate_random_bases(n)

qubits = encode_qubits(alice_bits, alice_bases)
bob_bits = measure_qubits(qubits, bob_bases)

# Simulate errors
bob_bits_noisy = introduce_errors(bob_bits, error_rate=0.1)

# Error detection
if error_detection(alice_bits, bob_bits_noisy):
    print("Key is mostly correct. Applying error correction...")
    bob_bits_fixed = correct_errors(alice_bits, bob_bits_noisy)
    print("Corrected Key:", bob_bits_fixed)
else:
    print("Key was too corrupted! Restarting the protocol.")


Error Check: 1/10 mismatched
Key is mostly correct. Applying error correction...
Corrected Key: [1, 1, 0, 0, 0, 1, 0, 0, 1, 1]
